<a href="https://colab.research.google.com/github/ameyshinde28/NLP_ChatBot/blob/main/NLP_Chat_Bot_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
import numpy as np


In [ ]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
stemmer = PorterStemmer()

In [ ]:
%%writefile NLP_Chat_Bot/nltk_utils.py

import nltk
from nltk.stem.porter import PorterStemmer
import numpy as np
nltk.download("punkt")


stemmer = PorterStemmer()

def tokenize(sentence):
  return nltk.word_tokenize(sentence)

def stem(word):
  return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, all_words):
  tokenized_sentence = [stem(w) for w in tokenized_sentence]

  bag = np.zeros(len(all_words), dtype=np.float32)
  for idx, w in enumerate(all_words):
    if w in tokenized_sentence:
      bag[idx] = 1.0
  return bag

Overwriting NLP_Chat_Bot/nltk_utils.py


In [ ]:
a = "How long does shipping take?"
print(a)
a = tokenize(a)
print(a)

How long does shipping take?


NameError: name 'tokenize' is not defined

In [ ]:
words = ["organize", "organizes", "organizing"]
stemmed_words = [stem(w) for w in words]
print(stemmed_words)

#### Train.py

In [ ]:
from pickle import TRUE
import requests
from pathlib import Path

nlp_chat_bot_path = Path("NLP_Chat_Bot/")
nlp_chat_bot_path.mkdir(parents=True, exist_ok = True)
intents_json_path = nlp_chat_bot_path / "intents.json"
if not intents_json_path.is_file():
  with open(intents_json_path, "wb") as f:
    request = requests.get("https://raw.githubusercontent.com/patrickloeber/python-fun/master/chatbot-gui/intents.json")
    print(f"Downloadning {intents_json_path}....")
    f.write(request.content)
else:
  print(f"{intents_json_path} already exists, skipping download")

NLP_Chat_Bot/intents.json already exists, skipping download


In [ ]:
%%writefile NLP_Chat_Bot/train.py
import json
from torch.utils.data import Dataset, DataLoader
import requests
from pathlib import Path
from nltk_utils import tokenize, stem, bag_of_words
import numpy as np
import os
from model import NeuralNet
import torch
import torch.nn as nn

nlp_chat_bot_path = Path("NLP_Chat_Bot/")
nlp_chat_bot_path.mkdir(parents=True, exist_ok = True)
intents_json_path = nlp_chat_bot_path / "intents.json"

with open(intents_json_path, "r") as f:
  intents = json.load(f)
  # print(intents)


all_words = []
tags =[]
xy = []

for intent in intents['intents']:
  # print(intent)
  tag = intent['tag']
  tags.append(tag)

  for pattern in intent['patterns']:
    w = tokenize(pattern)
    all_words.extend(w)
    xy.append((w, tag))

ignore_words = ["?", "!", ".", ","]

all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
print(all_words)
tags = sorted(set(tags))
print(tags)


X_train = []
y_train= []

for (pattern_sentence, tag) in xy:
  bag = bag_of_words(pattern_sentence, all_words)
  X_train.append(bag)

  label = tags.index(tag)
  y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)


class ChatDataset(Dataset):
  def  __init__(self):
    self.n_samples = len(X_train)
    self.x_data = X_train
    self.y_data = y_train

  def __getitem__(self, index):
    return self.x_data[index], self.y_data[index]

  def __len__(self):
    return self.n_samples

#Hyperparameters
BATCH_SIZE = 8
NUM_WORKERS = os.cpu_count()
hidden_size = 8
output_size = len(tags)
input_size = len(X_train[0])
learning_rate = 0.001
num_epochs = 1000



dataset = ChatDataset()
train_dataloader = DataLoader(dataset=dataset,
                              batch_size=BATCH_SIZE,
                              shuffle=True,
                              num_workers=NUM_WORKERS)

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
model = NeuralNet(input_size, hidden_size, output_size).to(device)


#loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr =learning_rate)

for epoch in range(num_epochs):
  for (words, labels) in train_dataloader:
    words = words.to(device)
    labels = labels.to(device)

    # forward pass
    outputs = model(words).to(device)
    loss = criterion(outputs, labels)

    # backward and optimizer step
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  if (epoch+1) % 100 == 0:
    print(f"epoch {epoch+1}/{num_epochs}, loss={loss.item():.4f}")

data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "output_size": output_size,
    "hidden_size": hidden_size,
    "all_words": all_words,
    "tags": tags
}

FILE = 'NLP_Chat_Bot/data.pth'
torch.save(data, FILE)

print(f"training complete, file saved to {FILE}")


Overwriting NLP_Chat_Bot/train.py


### MODEL

In [ ]:
%%writefile NLP_Chat_Bot/model.py
import torch
import torch.nn as nn

class NeuralNet(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super().__init__()
    self.l1 = nn.Linear(input_size, hidden_size)
    self.l2 = nn.Linear(hidden_size, hidden_size)
    self.l3 = nn.Linear(hidden_size, num_classes)
    self.relu = nn.ReLU()

  def forward(self, x):
    out = self.l1(x)
    out = self.relu(out)
    out = self.l2(out)
    out = self.relu(out)
    out = self.l3(out)

    return out

Overwriting NLP_Chat_Bot/model.py


In [ ]:
!python NLP_Chat_Bot/train.py

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
["'s", 'a', 'accept', 'anyon', 'are', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'deliveri', 'do', 'doe', 'funni', 'get', 'good', 'goodby', 'have', 'hello', 'help', 'hey', 'hi', 'how', 'i', 'is', 'item', 'joke', 'kind', 'know', 'later', 'long', 'lot', 'mastercard', 'me', 'my', 'of', 'onli', 'pay', 'paypal', 'see', 'sell', 'ship', 'someth', 'take', 'tell', 'thank', 'that', 'there', 'what', 'when', 'which', 'with', 'you']
['delivery', 'funny', 'goodbye', 'greeting', 'items', 'payments', 'thanks']
epoch 100/1000, loss=0.3927
epoch 200/1000, loss=0.0893
epoch 300/1000, loss=0.1611
epoch 400/1000, loss=0.0333
epoch 500/1000, loss=0.0049
epoch 600/1000, loss=0.0020
epoch 700/1000, loss=0.0050
epoch 800/1000, loss=0.0004
epoch 900/1000, loss=0.0003
epoch 1000/1000, loss=0.0014
training complete, file saved to NLP_Chat_Bot/data.pth


In [ ]:
%%writefile NLP_Chat_Bot/chat.py
import random
from random import choice
import json
import json
import torch
from model import NeuralNet
from nltk_utils import bag_of_words, tokenize

device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

with open("NLP_Chat_Bot/intents.json", 'r') as f:
  intents = json.load(f)

FILE = "NLP_Chat_Bot/data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data["all_words"]
tags = data["tags"]
model_state = data["model_state"]


model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

bot_name = "BOTY"
print("Let's chat! type 'quit' to exit")
while True:
  sentence = input("You: ")
  if sentence == "quit":
    break
  else:
    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output= model(X)
    _, predicted = torch.max(output, dim=1)
    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]

    if prob.item() > 0.75:
      for intent in intents["intents"]:
        if tag == intent["tag"]:
          response = random.choice(intent["responses"])
          print(f"{bot_name}: {response}")

Overwriting NLP_Chat_Bot/chat.py


In [ ]:
!python NLP_Chat_Bot/chat.py

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Let's chat! type 'quit' to exit
You: Hi
BOTY: Hello, thanks for visiting
You: What do you sell
BOTY: We sell coffee and tea
You: Time for shipping
You: How long takes shipping
BOTY: Delivery takes 2-4 days
You: 